In [2]:
import findspark
findspark.init()
import pyspark

In [3]:
spark = pyspark.sql.SparkSession.builder.appName('lreg').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load('../data/sample_linear_regression_data.txt')

In [14]:
# create Linear Regression model
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [15]:
lr_model = lr.fit(training)

In [16]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [17]:
lr_model.intercept

0.14228558260358093

In [18]:
training_summary = lr_model.summary

In [19]:
training_summary.r2

0.027839179518600154

In [20]:
training_summary.rootMeanSquaredError

10.16309157133015

### Train / Test Split

In [21]:
all_data = spark.read.format('libsvm').load('../data/sample_linear_regression_data.txt')

In [26]:
train, test = all_data.randomSplit([0.7, 0.3])

In [28]:
train.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                353|
|   mean|0.20180731956870454|
| stddev| 10.645427740335482|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [29]:
test.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                148|
|   mean| 0.3882656494049824|
| stddev|  9.524509591970393|
|    min|-22.949825936196074|
|    max| 22.647750304177556|
+-------+-------------------+



### Train Linear Regression & Evaluate on Test Set

In [30]:
better_model = lr.fit(train)

In [31]:
test_results = better_model.evaluate(test)

In [33]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-27.864605785279473|
|-22.736948471543347|
|-15.697155990044935|
|-16.149208564341514|
|-15.221714488592319|
| -16.19486054625837|
|-13.936988835008327|
|-15.819714445536047|
|-18.911222589027737|
| -18.53958098153997|
|-15.008632558700732|
|-17.022676659697673|
|-11.826497716309188|
|-12.378337627858029|
| -14.36367087208209|
| -8.747464407802973|
|-10.573583060679178|
| -13.99990205073908|
| -16.08082094512973|
| -14.47654917390615|
+-------------------+
only showing top 20 rows



In [34]:
test_results.rootMeanSquaredError

9.730435279696122

In [35]:
test_results.r2

-0.05080872552760307

### Deploying a Model

In [36]:
unlabeled_data = test.select('features')

In [38]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [39]:
predictions = better_model.transform(unlabeled_data)

In [40]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  4.9147798490833985|
|(10,[0,1,2,3,4,5,...|  1.3045607073775414|
|(10,[0,1,2,3,4,5,...| -3.4711366329188267|
|(10,[0,1,2,3,4,5,...|   -1.34499179254183|
|(10,[0,1,2,3,4,5,...|   -2.10500624408363|
|(10,[0,1,2,3,4,5,...| -0.8316317179511803|
|(10,[0,1,2,3,4,5,...| -2.0145237307862462|
|(10,[0,1,2,3,4,5,...|-0.04229488203451...|
|(10,[0,1,2,3,4,5,...|  3.1305375564044353|
|(10,[0,1,2,3,4,5,...|  3.2048135016176285|
|(10,[0,1,2,3,4,5,...| 0.24587430576960523|
|(10,[0,1,2,3,4,5,...|  2.6936981506222315|
|(10,[0,1,2,3,4,5,...| -2.0405901788495813|
|(10,[0,1,2,3,4,5,...| -0.7749979785075011|
|(10,[0,1,2,3,4,5,...|  1.3237428079774742|
|(10,[0,1,2,3,4,5,...|  -4.025762591448223|
|(10,[0,1,2,3,4,5,...| -1.8940733203536815|
|(10,[0,1,2,3,4,5,...|  2.1217345507722385|
|(10,[0,1,2,3,4,5,...|  4.2537479487371606|
|(10,[0,1,2,3,4,5,...|  3.044746